In [1]:
import os
import numpy as np
from PIL import Image, ImageEnhance
import random
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform, rotate

# Directories
original_dataset_dir = r'D:\Project\aug_data\paper_data'
augmented_dataset_dir = r'D:\Project\aug_data\data_aug'

# Create augmented dataset directory if not exists
os.makedirs(augmented_dataset_dir, exist_ok=True)

# Get all class folders in the original dataset
classes = [f for f in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, f))]

# Transformation parameters
shear_x = 0.02  # Shear amount along x-axis
shear_y = 0.02  # Shear amount along y-axis
horizontal_flip = True  # Apply horizontal flip
rotation_range = 20  # Rotation angle range (±20 degrees)
noise_variance = 0.02  # Variance for Gaussian noise
brightness_factor = (0.8, 1.2)  # Range for brightness adjustment
contrast_factor = (0.8, 1.2)  # Range for contrast adjustment

# Function to resize images
def resize_image(image_array, size):
    """Resize image to the specified size."""
    img = Image.fromarray((image_array * 255).astype(np.uint8))
    img_resized = img.resize(size)
    return np.array(img_resized) / 255.0

# Function to apply shear transformation
def shear_image(image_array, shear_x, shear_y):
    """Apply shear transformation to the image."""
    shear_matrix = np.array([[1, shear_x, 0], [shear_y, 1, 0], [0, 0, 1]])
    sheared_image = np.zeros_like(image_array)
    for i in range(image_array.shape[2]):
        sheared_image[:, :, i] = affine_transform(image_array[:, :, i], shear_matrix[:2, :2], order=1, mode='constant', cval=0)
    return sheared_image

# Function to apply horizontal flip
def flip_image(image_array):
    """Apply horizontal flip to the image."""
    return np.flip(image_array, axis=1)

# Function to apply rotation
def rotate_image(image_array, angle):
    """Rotate image by a given angle."""
    rotated_image = np.zeros_like(image_array)
    for i in range(image_array.shape[2]):
        rotated_image[:, :, i] = rotate(image_array[:, :, i], angle, reshape=False, order=1, mode='constant', cval=0)
    return rotated_image

# Function to add Gaussian noise
def add_noise(image_array, variance):
    """Add Gaussian noise to the image."""
    noise = np.random.normal(0, variance ** 0.5, image_array.shape)
    noisy_image = np.clip(image_array + noise, 0, 1)  # Clip to valid range
    return noisy_image

# Function to adjust brightness and contrast
def adjust_brightness_contrast(image, brightness_range, contrast_range):
    """Adjust brightness and contrast of an image."""
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer_brightness = ImageEnhance.Brightness(img)
    img = enhancer_brightness.enhance(random.uniform(*brightness_range))
    enhancer_contrast = ImageEnhance.Contrast(img)
    img = enhancer_contrast.enhance(random.uniform(*contrast_range))
    return np.array(img) / 255.0

# Process each class folder
for class_name in classes:
    class_path = os.path.join(original_dataset_dir, class_name)
    
    # Create class folder in the augmented dataset directory
    save_class_path = os.path.join(augmented_dataset_dir, class_name)
    os.makedirs(save_class_path, exist_ok=True)

    # Get all image files in the class directory
    images = [img for img in os.listdir(class_path) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not images:
        print(f"Skipping {class_name}: No images found.")
        continue

    for image_name in images:
        image_path = os.path.join(class_path, image_name)

        # Open the image and normalize it
        with Image.open(image_path) as img:
            img = img.convert("RGB")
            original_image = np.array(img) / 255.0

            # Resize to 128x128
            resized_128x128 = resize_image(original_image, (128, 128))

            # Apply augmentations
            sheared_image = shear_image(resized_128x128, shear_x, shear_y)
            flipped_image = flip_image(sheared_image) if horizontal_flip else sheared_image
            rotated_image = rotate_image(resized_128x128, random.uniform(-rotation_range, rotation_range))
            noisy_image = add_noise(resized_128x128, noise_variance)
            bright_contrast_image = adjust_brightness_contrast(resized_128x128, brightness_factor, contrast_factor)

            # Convert back to PIL images
            resized_pil = Image.fromarray((resized_128x128 * 255).astype(np.uint8))
            sheared_pil = Image.fromarray((sheared_image * 255).astype(np.uint8))
            flipped_pil = Image.fromarray((flipped_image * 255).astype(np.uint8))
            rotated_pil = Image.fromarray((rotated_image * 255).astype(np.uint8))
            noisy_pil = Image.fromarray((noisy_image * 255).astype(np.uint8))
            bright_contrast_pil = Image.fromarray((bright_contrast_image * 255).astype(np.uint8))

            # Save augmented images
            base_name = os.path.splitext(image_name)[0]
            resized_pil.save(os.path.join(save_class_path, f"{base_name}_resized.png"))
            sheared_pil.save(os.path.join(save_class_path, f"{base_name}_sheared.png"))
            flipped_pil.save(os.path.join(save_class_path, f"{base_name}_flipped.png"))
            rotated_pil.save(os.path.join(save_class_path, f"{base_name}_rotated.png"))
            noisy_pil.save(os.path.join(save_class_path, f"{base_name}_noisy.png"))
            bright_contrast_pil.save(os.path.join(save_class_path, f"{base_name}_bright_contrast.png"))

    print(f"Augmentation completed for class: {class_name}")

print("All augmented images saved in:", augmented_dataset_dir)


Augmentation completed for class: Amanita citrina
Augmentation completed for class: Amanita muscaria
Augmentation completed for class: Amanita rubescens
Augmentation completed for class: Cantharellus cibarius
Augmentation completed for class: Sarcoscypha austriaca
Augmentation completed for class: Stropharia aeruginosa
Augmentation completed for class: Verpa bohemica
All augmented images saved in: D:\Project\aug_data\data_aug
